In [2]:
import yfinance as yf
import pandas as pd
from google.colab import drive
import os
from datetime import datetime
import pytz
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#timezones
tz_us = "America/New_York"
tz_hk = "Asia/Hong_Kong"
tz_jp = "Asia/Tokyo"
tz_kr = "Asia/Seoul"
tz_sg = "Asia/Singapore"
tz_uk = "Europe/London"
tz_gr = "Europe/Berlin"
tz_am = "Europe/Amsterdam"
tz_fr = "Europe/Paris"

#trading hours
us_hrs = [[9,30],[4,0]]
hk_hrs = [[9,30],[4,0]]
jp_hrs = [[9,0],[3,0]]
kr_hrs = [[9,0],[3,30]]
sg_hrs = [[9,0],[5,0]]
uk_hrs = [[8,0],[4,30]]
gr_hrs = [[9,0],[5,30]]
am_hrs = [[9,0],[5,30]]
fr_hrs = [[9,0],[5,30]]

def convert_dt(prices, timezone, hrs):
  tz = pytz.timezone(timezone)
  prices['Date'] = pd.to_datetime(prices['Date'])
  prices['Date'] = prices['Date'].apply(lambda x: pd.Timestamp(x).replace(hour=hrs[0][0], minute=hrs[0][1]))
  #prices['Date'] = prices['Date'].dt.tz_localize(tz=tz,ambiguous='infer')
  prices['Date'] = prices['Date'].dt.tz_convert(pytz.utc)
  return prices


In [3]:
# List of tickers
tickers = ["AAPL", "MSFT", "GOOGL"]  # Add your desired tickers here

# Create a directory to store the CSV files
output_directory = "/content/drive/MyDrive/findata/data"
os.makedirs(output_directory, exist_ok=True)

# Create a directory to store the tracking file
tracking_directory = "/content/drive/MyDrive/findata/log"
os.makedirs(tracking_directory, exist_ok=True)

# Create a tracking CSV file
tracking_file = os.path.join(tracking_directory, "download_dates.csv")
tracking_data = {}  # Dictionary to store tracking data

# Load existing tracking data if the file exists
if os.path.exists(tracking_file):
    existing_data = pd.read_csv(tracking_file)
    for index, row in existing_data.iterrows():
        tracking_data[row["Ticker"]] = row["Download_Date"]

# Function to download and save data
def download_and_save_data(ticker):
    start_date = "2020-01-01"
    end_date = "2022-12-31"

    # Download stock price data
    data = yf.download(ticker, start=start_date, end=end_date)

    # Group data by year
    data_by_year = data.groupby(data.index.year)

    # Create a folder for each ticker
    ticker_directory = os.path.join(output_base_directory, ticker)
    os.makedirs(ticker_directory, exist_ok=True)

    # Save data into separate CSV files for each year in the ticker folder
    for year, year_data in data_by_year:
        year_file_name = os.path.join(ticker_directory, f"{year}.csv")
        year_data.to_csv(year_file_name)

    # Update or add download date to tracking data
    tracking_data[ticker] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Write updated tracking data to the tracking CSV file
    with open(tracking_file, "w") as f:
        f.write("Ticker,Download_Date\n")
        for t, date in tracking_data.items():
            f.write(f"{t},{date}\n")

# Download and save data for each ticker
for ticker in tickers:
    download_and_save_data(ticker)

print("Data downloaded and saved in separate CSV files by ticker and by year.")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Data downloaded and saved in separate CSV files by ticker and by year.


In [ ]:
# functions for adding new tickers
def get_eod_yahoo(ticker, period="max", start=None):
  """Retrieves eod data from yahoo and converts date column to datetime object"""
  name = yf.Ticker(ticker)
  if not start:
    prices = name.history(period=period)
  else:
    prices = name.history(start=start)
  prices = prices.reset_index()
  return prices

# def get_eod_quandl():
#   return

def save_details(prices):
  prices['Year'] = prices['Date'].dt.strftime('%Y')
  prices['Year'] = prices['Year'].astype(int)
  min = int(prices['Year'].min())
  max = int(prices['Year'].max())
  return prices, min, max

def write_eod(ticker, prices):
  prices, min, max = save_details(prices)
  initial = ticker[0].lower()

  #data saving
  for i in range(min, max+1):
    eod = prices.query('Year == @i')
    eod = eod.drop('Year', axis=1)
    path = f'/content/drive/MyDrive/data/eod/{i}/{initial}'
    try:
      os.makedirs(f'{path}/')
      eod.to_csv(f'{path}/{ticker}.csv')
    except:
      eod.to_csv(f'{path}/{ticker}.csv')
  return

In [ ]:
#functions for updating existing tickers

def append_eod(ticker,prices):
  prices, min, max = save_details(prices)
  initial = ticker[0].lower()

  if min != max:
    eod = prices.query('Year == @min')
    eod.drop('Year', axis=1)
    path = f'/content/drive/MyDrive/data/eod/{min}/{initial}'
    eod.to_csv(f'{path}/{ticker}.csv', mode='a', headers=False)

    eod = prices.query('Year == @min')
    eod.drop('Year', axis=1)
    path = f'/content/drive/MyDrive/data/eod/{max}/{initial}'
    eod.to_csv(f'{path}/{ticker}.csv', mode='a', headers=False)

  else:
    prices.drop('Year', axis=1)
    path = f'/content/drive/MyDrive/data/eod/{min}/{initial}'
    try:
      os.makedirs(f'{path}/')
      prices.to_csv(f'{path}/{ticker}.csv')
    except:
      prices.to_csv(f'{path}/{ticker}.csv')

def update_log(log):
  path = f'/content/drive/MyDrive/data/eod/log.csv'
  if os.path.isfile(path):
    current = pd.read_csv(path)
    current = pd.concat([log, current])
    current = current.drop_duplicates('Ticker').sort_values(by='Ticker',ascending=True)
    current.to_csv(path)
  else:
    log.sort_values(by='Ticker',ascending=True).to_csv(path)
  return

In [ ]:
#functions for retrieving data
def fetch_eod(ticker, which, start, end=None):
  if end == None:
    current_datetime=datetime.datetime.now()
    end = current_datetime.strftime("%d/%m/%Y")
  start_year = int(start[-4:])
  end_year = int(end[-4:])
  initial = ticker[0].lower()
  prices = pd.DataFrame(columns=['Date', which])

  for i in range(start_year, end_year+1):
    path = f'/content/drive/MyDrive/data/eod/{i}/{initial}/{ticker}.csv'
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])

    if i == start_year:
      df = df[(df['Date'] > datetime.datetime.strptime(start, "%d/%m/%Y").replace(tzinfo=utc))]
      prices = df[['Date', which]]
    elif i == end_year:
      df = df[(df['Date'] < datetime.datetime.strptime(end, "%d/%m/%Y").replace(tzinfo=utc))]
      prices = pd.concat([prices, df[['Date', which]]], axis=0)
    else:
      prices = pd.concat([prices, df[['Date', which]]], axis=0)
  return prices

In [ ]:
# run this cell to add new tickers
ticker_path = f'/content/drive/MyDrive/data/eod/Additions.csv'
tickers = pd.read_csv(ticker_path)
# tickers['Source']='yahoo'
tz = tz_us
hrs = us_hrs

symbol = []
data = []
date = []

#get and save eod data
for ticker, source in zip(list(tickers['Symbol']),list(tickers['Source'])):
  if source == 'yahoo':
    prices = get_eod_yahoo(ticker, period="max")

  elif source == 'quandl':
    continue
  prices = convert_dt(prices, timezone=tz, hrs=hrs)
  write_eod(ticker, prices)
  symbol.append(ticker)
  data.append(source)
  #record the date of latest update
  date.append(prices['Date'].max().strftime('%d/%m/%Y'))

#update log file
log = pd.DataFrame(zip(symbol, data, date), columns=['Ticker','Source','Last Update'])
update_log(log)

In [ ]:
# run this cell to update current tickers
path = f'/content/drive/MyDrive/data/eod/log.csv'
current = pd.read_csv("path")
i=0

#get and save eod data
for ticker, source, date in zip(current['Ticker'], current['Source'], current['Last Update']):
  if source =='yahoo':
    prices = get_eod_yahoo(ticker,start=date)
  elif source == 'quandl':
    pass
  save_eod(ticker, prices)
  #record the date of latest update
  current['Last Update'][i] = prices['Date'].max().strftime('%d/%m/%Y')
  i+=1

#update log file
current.to_csv(path)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-06-16 00:00:00+08:00,0.835704,0.883459,0.778399,0.792725,2198875000,0.0,0.0
2004-06-17 00:00:00+08:00,0.792725,0.835704,0.787950,0.807052,419007500,0.0,0.0
2004-06-18 00:00:00+08:00,0.802276,0.811827,0.754522,0.768848,182990000,0.0,0.0
2004-06-21 00:00:00+08:00,0.783174,0.787950,0.754522,0.764073,114085000,0.0,0.0
2004-06-22 00:00:00+08:00,0.764073,0.764073,0.764073,0.764073,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-13 00:00:00+08:00,350.000000,350.600006,347.000000,349.799988,27315463,0.0,0.0
2023-07-14 00:00:00+08:00,354.799988,354.799988,348.799988,352.600006,21213481,0.0,0.0
2023-07-18 00:00:00+08:00,351.799988,351.799988,335.000000,336.399994,29147987,0.0,0.0


In [ ]:
# run this cell to retrieve eod data
ticker_path = f'/content/drive/MyDrive/data/eod/Additions.csv'
which = 'Close'
start = '21/02/2001'
ticker_list = pd.read_csv(ticker_path)
data = pd.DataFrame()

for i, ticker in enumerate(ticker_list['Symbol']):
  prices = fetch_eod(ticker , which=which, start=start)
  prices = prices.rename(columns={'Close': ticker})
  if i==0:
    data = prices
  else:
    data = pd.merge(data, prices, on=['Date'])

data
#need to handle cases where the date i want to retrieve is earlier than the earliest date

FileNotFoundError: ignored

,Date,Close
34,2001-02-21 00:00:00-05:00,0.286477
35,2001-02-22 00:00:00-05:00,0.285528
36,2001-02-23 00:00:00-05:00,0.285528
37,2001-02-26 00:00:00-05:00,0.295962
38,2001-02-27 00:00:00-05:00,0.294065
...,...,...
133,2023-07-17 00:00:00-04:00,193.990005
134,2023-07-18 00:00:00-04:00,193.729996
135,2023-07-19 00:00:00-04:00,195.100006
136,2023-07-20 00:00:00-04:00,193.130005
